<a href="https://colab.research.google.com/github/ilhamazhar1308/Rock-Type-Classification-Using-Deep-Learning-Approach-Based-on-ResNet-34-Architecture/blob/main/DeepLearningGeofisika_Resnet34_Kelompok_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Mounted at /content/drive
Device: cuda


In [ ]:
data_dir = "/content/drive/MyDrive/Rocks"
full_ds = datasets.ImageFolder(root=data_dir)

class_name = full_ds.classes
num_classes = len(class_name)

print("Jumlah kelas:", num_classes)
print("Nama kelas:", class_name)
print("Total gambar:", len(full_ds))


Jumlah kelas: 53
Nama kelas: ['Amphibolite', 'Andesite', 'Anthracite', 'Basalt', 'Blueschist', 'Breccia', 'Carbonatite', 'Chalk', 'Chert', 'Coal', 'Conglomerate', 'Diamictite', 'Dolomite', 'Eclogite', 'Evaporite', 'Flint', 'Gabbro', 'Gneiss', 'Granite', 'Granulite', 'Greenschist', 'Greywacke', 'Hornfels', 'Komatiite', 'Limestone', 'Marble', 'Migmatite', 'Mudstone', 'Obsidian', 'Oil_shale', 'Oolite', 'Pegmatite', 'Phyllite', 'Porphyry', 'Pumice', 'Pyroxenite', 'Quartz_diorite', 'Quartz_monzonite', 'Quartzite', 'Quartzolite', 'Rhyolite', 'Sandstone', 'Scoria', 'Serpentinite', 'Shale', 'Siltstone', 'Slate', 'Talc_carbonate', 'Tephrite', 'Travertine', 'Tuff', 'Turbidite', 'Wackestone']
Total gambar: 2343


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_tfms = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(0.3,0.3,0.3),
    transforms.RandomPerspective(0.3, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

test_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])


In [ ]:
train_size = int(0.8 * len(full_ds))
test_size  = len(full_ds) - train_size

train_idx, test_idx = torch.utils.data.random_split(
    full_ds, [train_size, test_size]
)

train_ds = datasets.ImageFolder(data_dir, transform=train_tfms)
test_ds  = datasets.ImageFolder(data_dir, transform=test_tfms)

train_ds.samples = [full_ds.samples[i] for i in train_idx.indices]
test_ds.samples  = [full_ds.samples[i] for i in test_idx.indices]

print("Train:", len(train_ds))
print("Test :", len(test_ds))


Train: 1874
Test : 469


In [ ]:
batch_size = 32

train_dl = DataLoader(
    train_ds, batch_size=batch_size,
    shuffle=True, num_workers=2, pin_memory=True
)

test_dl = DataLoader(
    test_ds, batch_size=batch_size,
    shuffle=False, num_workers=2, pin_memory=True
)


In [ ]:
model = models.resnet34(
    weights=models.ResNet34_Weights.IMAGENET1K_V1
)

# Freeze
for param in model.parameters():
    param.requires_grad = False

# Ganti FC
model.fc = nn.Sequential(
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)

# Fine tuning layer terakhir
for name, param in model.named_parameters():
    if "layer4" in name:
        param.requires_grad = True

model = model.to(device)


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:00<00:00, 169MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.15)

optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4, weight_decay=1e-3
)


In [ ]:
def accuracy(logits, y):
    return (logits.argmax(1) == y).float().mean().item()


In [ ]:
def train_one_epoch(model, loader):
    model.train()
    loss_sum, acc_sum = 0, 0

    for x, y in tqdm(loader, desc="Train", leave=False):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item() * x.size(0)
        acc_sum  += accuracy(out, y) * x.size(0)

    return loss_sum / len(loader.dataset), acc_sum / len(loader.dataset)


In [ ]:
ckpt_dir = "/content/drive/MyDrive/Checkpoints_Rocks_FINAL"
os.makedirs(ckpt_dir, exist_ok=True)

epochs = 125
for ep in range(1, epochs + 1):
    loss, acc = train_one_epoch(model, train_dl)

    print(f"Epoch {ep:03d} | Loss {loss:.4f} | Acc {acc*100:.2f}%")

    if ep % 5 == 0:
        torch.save({
            "epoch": ep,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict()
        }, f"{ckpt_dir}/checkpoint_{ep}.pth")

        print("✅ Checkpoint saved")


Epoch 001 | Loss 4.0123 | Acc 3.95%


Epoch 002 | Loss 3.6804 | Acc 13.13%


Epoch 003 | Loss 3.4883 | Acc 18.78%


Epoch 004 | Loss 3.3498 | Acc 23.37%


Epoch 005 | Loss 3.2413 | Acc 26.31%
✅ Checkpoint saved


...

Epoch 121 | Loss 1.4436 | Acc 85.17%


Epoch 122 | Loss 1.4395 | Acc 85.38%


Epoch 123 | Loss 1.4397 | Acc 85.65%


Epoch 124 | Loss 1.4489 | Acc 84.85%


Epoch 125 | Loss 1.4409 | Acc 86.07%
✅ Checkpoint saved


In [ ]:
ckpt_dir = "/content/drive/MyDrive/Checkpoints_Rocks_FINAL"

def extract_epoch(fname):
    return int(re.search(r"checkpoint_(\d+)\.pth", fname).group(1))

checkpoint_files = [
    f for f in os.listdir(ckpt_dir) if f.endswith(".pth")
]

assert len(checkpoint_files) > 0, "Checkpoint tidak ditemukan!"

checkpoint_files.sort(key=extract_epoch)

latest_ckpt = os.path.join(ckpt_dir, checkpoint_files[-1])
print("Memuat checkpoint:", latest_ckpt)

checkpoint = torch.load(latest_ckpt, map_location=device)

model.load_state_dict(checkpoint["model"])
optimizer.load_state_dict(checkpoint["optimizer"])

start_epoch = checkpoint["epoch"] + 1
print("Lanjut dari epoch:", start_epoch)

epochs = 70

for ep in range(start_epoch, epochs + 1):
    loss, acc = train_one_epoch(model, train_dl)

    print(f"Epoch {ep:03d} | Loss {loss:.4f} | Acc {acc*100:.2f}%")

    if ep % 5 == 0:
        torch.save({
            "epoch": ep,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict()
        }, f"{ckpt_dir}/checkpoint_{ep}.pth")

        print("✅ Checkpoint saved")


Memuat checkpoint: /content/drive/MyDrive/Checkpoints_Rocks_FINAL/checkpoint_125.pth
Lanjut dari epoch: 126


In [ ]:
ckpt = torch.load(f"{ckpt_dir}/checkpoint_{epochs}.pth",
                  map_location=device)

model.load_state_dict(ckpt["model"])
print("Checkpoint loaded, siap prediksi")


Checkpoint loaded, siap prediksi


In [ ]:
def extract_epoch(fname):
    return int(re.search(r"checkpoint_(\d+)\.pth", fname).group(1))

ckpt_files = [f for f in os.listdir(ckpt_dir) if f.endswith(".pth")]
ckpt_files.sort(key=extract_epoch)

latest_ckpt = os.path.join(ckpt_dir, ckpt_files[-1])
print("Loading:", latest_ckpt)

ckpt = torch.load(latest_ckpt, map_location=device)
model.load_state_dict(ckpt["model"])

print("✅ Checkpoint terakhir dimuat, siap prediksi")


Loading: /content/drive/MyDrive/Checkpoints_Rocks_FINAL/checkpoint_125.pth
✅ Checkpoint terakhir dimuat, siap prediksi


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F
import os

def predict_image_with_photo(img_path, topk=5):
    assert os.path.exists(img_path), "File gambar tidak ditemukan"

    model.eval()
    img = Image.open(img_path).convert("RGB")

    # Transform
    x = test_tfms(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(x)
        probs = F.softmax(logits, dim=1)[0]


    topk_vals, topk_idx = probs.topk(topk)


    plt.figure(figsize=(6,6))
    plt.imshow(img)
    plt.axis("off")

    plt.title(
        f"PREDIKSI: {class_name[topk_idx[0]]}\n"
        f"CONFIDENCE: {topk_vals[0]*100:.2f}%"
    )

    plt.show()


    print("TOP PREDIKSI:")
    for i, v in zip(topk_idx, topk_vals):
        print(f"{class_name[i]:<25} {v*100:.2f}%")

    if topk_vals[0] < 0.70:
        print("⚠️ Model kurang yakin")

    return class_name[topk_idx[0]]


In [ ]:
predict_image_with_photo(
    "/content/drive/MyDrive/DEEPLEARN/testbatu3.jpg"
)


AssertionError: File gambar tidak ditemukan